In [1]:
import pandas as pd
import random
import math

In [2]:
csv = pd.read_csv('data.csv')


In [3]:
df = csv.groupby(['Country_Region'])['Confirmed', 'Deaths', 'Recovered'].sum().reset_index()
df = df[["Country_Region", "Confirmed", "Deaths", "Recovered"]]

<ipython-input-3-8454d5381422>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df = csv.groupby(['Country_Region'])['Confirmed', 'Deaths', 'Recovered'].sum().reset_index()


In [4]:
df_scaled = df.copy()
# df_scaled['Confirmed'].plot(kind = 'bar')
df_scaled['Confirmed'] = (df_scaled['Confirmed'] - df_scaled['Confirmed'].min()) / (df_scaled['Confirmed'].max() - df_scaled['Confirmed'].min())
df_scaled['Deaths'] = (df_scaled['Deaths'] - df_scaled['Deaths'].min()) / (df_scaled['Deaths'].max() - df_scaled['Deaths'].min())
df_scaled['Recovered'] = (df_scaled['Recovered'] - df_scaled['Recovered'].min()) / (df_scaled['Recovered'].max() - df_scaled['Recovered'].min())


In [5]:
som_dim = (10, 10) # x, y dimensions
learning_rate = 0.05
iterations = 1000

radius = max(som_dim) // 2
data_dimensions = df_scaled.shape[1] - 1 #remove index from shape
time_const = iterations / math.log(radius)

neurons = [[] for _ in range(som_dim[0])]

for row in range(som_dim[0]):
    for col in range(som_dim[1]):
        neurons[row].append([random.random() for _ in range(data_dimensions)])


In [7]:
for m_iter in range(iterations):
    input_vec = df_scaled.sample()
    input_vec = input_vec.values.tolist()[0][1:]
    
    dist = [] #list of tuples ( (i,j), distance )

    for row in range(som_dim[0]):
        for col in range(som_dim[1]):
            curr_neuron = neurons[row][col]
            m_dist = math.dist(input_vec, curr_neuron)
            dist.append( ((row, col),m_dist))
    
    neuron_ij, dist = min(dist, key=lambda x: x[1]))



((5, 6), 0.12248235268433319)
